In [39]:
# frontend.py - Lottery 合约前端
from web3 import Web3
import json

# 連接到本地以太坊節點
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
assert w3.is_connected()

print("✅ 已連接到以太坊節點")
print(f"Chain ID: {w3.eth.chain_id}")
print(f"可用帳戶: {w3.eth.accounts}")

# load Lottery ABI
with open("build/Lottery.abi") as f:
    abi = json.load(f)

# Lottery 合约地址 (load from lottery_deploy.ipynb)
contract_address = Web3.to_checksum_address("0x6C48Cbb9cDE86508A5885Fa3d5603964Dd2AD354")

lottery = w3.eth.contract(address=contract_address, abi=abi)
print(f"✅ 已加載 Lottery 合約，地址: {contract_address}")

✅ 已連接到以太坊節點
Chain ID: 11330023
可用帳戶: ['0x02f396aD6Ba37602441bE148AC2876ff595E2aea', '0x287665Ae662148dA1a77eE7ec6D5fF6170E9093f', '0xF4A24c01ED89684A087254E88D97A0eaF99866E7', '0x6F3019f6B620E03fDDB14bb6fFbB08a05263E8dd', '0x46f03e16aEd64af7CC8fcFa221221E702a85bF1a']
✅ 已加載 Lottery 合約，地址: 0x6C48Cbb9cDE86508A5885Fa3d5603964Dd2AD354


In [28]:
def check_account_balance(account):
    balance = w3.eth.get_balance(account)
    print(f"address:{account} balance: {w3.from_wei(balance, 'ether')} ETH")

In [60]:
def display_lottery_status():
    # ============================================
    # 查看合约狀態
    # ============================================
    print("\n" + "="*50)
    print("查看 Lottery 合約狀態")
    print("="*50)

    try:
        entry_fee = lottery.functions.entryFee().call()
        participant_count = lottery.functions.getParticipantCount().call()
        is_active = lottery.functions.isLotteryActive().call()
        prize_pool = lottery.functions.prizePool().call()
        owner = lottery.functions.owner().call()
        contract_balance = lottery.functions.getContractBalance().call()
        
        print(f"合约擁有者: {owner}")
        print(f"入場費: {w3.from_wei(entry_fee, 'ether')} ETH ({entry_fee} wei)")
        print(f"參與者數量: {participant_count}")
        print(f"抽獎狀態: {'✅ 進行中' if is_active else '❌ 已暫停'}")
        print(f"獎金池: {w3.from_wei(prize_pool, 'ether')} ETH")
        print(f"合約餘額: {w3.from_wei(contract_balance, 'ether')} ETH")
        
        # 查看上一輪獲勝者（如果有）
        last_winner = lottery.functions.lastWinner().call()
        last_prize = lottery.functions.lastPrize().call()
        if last_winner != "0x0000000000000000000000000000000000000000":
            print(f"\n上一輪獲勝者: {last_winner}")
            print(f"上一輪獎金: {w3.from_wei(last_prize, 'ether')} ETH")
        
    except Exception as e:
        print(f"❌ 讀取合約狀態失敗: {e}")
        exit(1)

In [61]:
def display_participants():
    # ============================================
    # 查看參與者列表
    # ============================================
    print("\n" + "="*50)
    print("查看參與者列表")
    print("="*50)

    try:
        participants = lottery.functions.getParticipants().call()
        if len(participants) > 0:
            print(f"參與者列表 ({len(participants)} 人):")
            for i, participant in enumerate(participants):
                print(f"  {i+1}. {participant}")
        else:
            print("目前沒有參與者")
    except Exception as e:
        print(f"❌ 讀取參與者列表失敗: {e}")

In [51]:
def participate_in_lottery(account):
    # ============================================
    # 3. 參與抽獎
    # ============================================
    print("\n" + "="*50)
    print("參與抽獎")
    print("="*50)

    balance = w3.eth.get_balance(account)
    entry_fee = lottery.functions.entryFee().call()
    is_active = lottery.functions.isLotteryActive().call()
    try:
        already_participated = lottery.functions.isParticipant(account).call()
        
        if already_participated:
            print("⚠️  您已經參與過本輪抽獎")
        elif not is_active:
            print("⚠️  抽獎目前暫停中，無法參與")
        else:
            # 檢查餘額是否足夠
            if balance < entry_fee:
                print(f"❌ 餘額不足！需要 {w3.from_wei(entry_fee, 'ether')} ETH，但只有 {w3.from_wei(balance, 'ether')} ETH")
            else:
                print(f"準備參與抽獎，需要支付 {w3.from_wei(entry_fee, 'ether')} ETH...")
                
                # 發送交易參與抽獎
                tx_hash = lottery.functions.enter().transact({
                    "from": account,
                    "value": entry_fee,
                    "gas": 100_000
                })
                
                print(f"交易哈希: {tx_hash.hex()}")
                
                # 等待交易確認
                receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
                
                if receipt.status == 1:
                    print(f"✅ 參與成功！交易已確認，區塊: {receipt.blockNumber}")
                    
                    # 再次查看參與者數量
                    participant_count = lottery.functions.getParticipantCount().call()
                    prize_pool = lottery.functions.prizePool().call()
                    print(f"現在參與者數量: {participant_count}")
                    print(f"現在獎金池: {w3.from_wei(prize_pool, 'ether')} ETH")
                else:
                    print("❌ 交易失敗")
                    
    except Exception as e:
        print(f"❌ 參與抽獎失敗: {e}")
        print("提示: 請確保賬戶已解鎖（在 geth console 中執行: personal.unlockAccount(eth.accounts[i], passwd, 600)）")

In [49]:
def draw_lottery(account):    
    # ============================================
    # 4. 抽獎（僅合約擁有者）
    # ============================================
    print("\n" + "="*50)
    print("執行抽獎（僅合約擁有者）")
    print("="*50)
    is_active = lottery.functions.isLotteryActive().call()
    try:
        owner = lottery.functions.owner().call()
        participant_count = lottery.functions.getParticipantCount().call()
        
        if account.lower() != owner.lower():
            print(f"⚠️  只有合約擁有者可以執行抽獎")
            print(f"   合約擁有者: {owner}")
            print(f"   當前賬戶: {account}")
        elif participant_count == 0:
            print("⚠️  目前沒有參與者，無法抽獎")
        elif not is_active:
            print("⚠️  抽獎已暫停，無法執行")
        else:
            print(f"準備執行抽獎，目前有 {participant_count} 位參與者...")
            
            tx_hash = lottery.functions.drawLottery().transact({
                "from": account,
                "gas": 200_000
            })
            
            print(f"交易哈希: {tx_hash.hex()}")
            
            receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
            
            if receipt.status == 1:
                print(f"✅ 抽獎完成！交易已確認，區塊: {receipt.blockNumber}")
                
                # 查看獲勝者
                winner = lottery.functions.lastWinner().call()
                prize = lottery.functions.lastPrize().call()
                print(f"\n🎉 獲勝者: {winner}")
                print(f"💰 獎金: {w3.from_wei(prize, 'ether')} ETH")
                
                # 查看新的狀態
                participant_count = lottery.functions.getParticipantCount().call()
                prize_pool = lottery.functions.prizePool().call()
                print(f"\n新一輪狀態:")
                print(f"  參與者數量: {participant_count}")
                print(f"  獎金池: {w3.from_wei(prize_pool, 'ether')} ETH")
            else:
                print("❌ 抽獎失敗")
                
    except Exception as e:
        print(f"❌ 執行抽獎失敗: {e}")


In [50]:
def list_management_functions(account):
    # ============================================
    # 5. 管理功能（僅合約擁有者）
    # ============================================
    print("\n" + "="*50)
    print("管理功能（僅合約擁有者）")
    print("="*50)
    try:
        owner = lottery.functions.owner().call()
        
        if account.lower() != owner.lower():
            print(f"⚠️  只有合約擁有者可以使用管理功能")
        else:
            print("可用管理功能:")
            print("  - setLotteryActive(bool): 暫停/啟動抽獎")
            print("  - setEntryFee(uint256): 修改入場費（僅在沒有參與者時）")
            print("  - manualReset(): 手動重置抽獎")
            print("  - emergencyWithdraw(): 緊急提取合約餘額")
            
            # 示例：暫停/啟動抽獎
            current_status = lottery.functions.isLotteryActive().call()
            print(f"\n當前抽獎狀態: {'進行中' if current_status else '已暫停'}")
            # 如果需要修改狀態，可以取消下面的註釋
            # new_status = not current_status
            # tx_hash = lottery.functions.setLotteryActive(new_status).transact({
            #     "from": account,
            #     "gas": 100_000
            # })
            # receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
            # print(f"狀態已更改為: {'進行中' if new_status else '已暫停'}")
            
    except Exception as e:
        print(f"❌ 讀取管理信息失敗: {e}")

    print("\n" + "="*50)
    print("完成！")
    print("="*50)

In [55]:
official_account = w3.eth.accounts[0] # 合約擁有者帳戶, nycu
user1_account = w3.eth.accounts[2]    # 一般使用者帳戶, user1
user2_account = w3.eth.accounts[3]    # 另一個使用者帳戶, user2
user3_account = w3.eth.accounts[4]    # 另一個使用者帳戶, user3

In [56]:
check_account_balance(official_account)
check_account_balance(user1_account)
check_account_balance(user2_account)
check_account_balance(user3_account)

address:0x02f396aD6Ba37602441bE148AC2876ff595E2aea balance: 43119.107234553254238034 ETH
address:0xF4A24c01ED89684A087254E88D97A0eaF99866E7 balance: 1.009811997998683986 ETH
address:0x6F3019f6B620E03fDDB14bb6fFbB08a05263E8dd balance: 0.979875367999127576 ETH
address:0x46f03e16aEd64af7CC8fcFa221221E702a85bF1a balance: 1.009870337999092366 ETH


In [57]:
# 修改entry fee為0.01 ether
# lottery.functions.setEntryFee(w3.to_wei(0.01, 'ether')).transact({'from': official_account, 'gas': 100_000})

In [58]:
# 記得先手動解鎖帳戶
# personal.unlockAccount(eth.accounts[4], "user3", 0)
join_accounts = [user1_account, user2_account, user3_account]
for join_account in join_accounts:
    participate_in_lottery(join_account)


參與抽獎
準備參與抽獎，需要支付 0.01 ETH...
交易哈希: 3d3b308aab0053e0679b92f2704a2823c8889b6c00da65fe57e8c1fa4bcefe3f
✅ 參與成功！交易已確認，區塊: 24861
現在參與者數量: 1
現在獎金池: 0.01 ETH

參與抽獎
準備參與抽獎，需要支付 0.01 ETH...
交易哈希: 763e0cf33619d626d57730c8a60ac671a3670e56e832b76626257ef105523195
✅ 參與成功！交易已確認，區塊: 24862
現在參與者數量: 2
現在獎金池: 0.02 ETH

參與抽獎
準備參與抽獎，需要支付 0.01 ETH...
交易哈希: eebb37e0d04aaed8b701df7aba5d904964e8be8a03c66d4f34fcb1d5cd69f083
✅ 參與成功！交易已確認，區塊: 24863
現在參與者數量: 3
現在獎金池: 0.03 ETH


In [62]:
display_lottery_status()
display_participants()


查看 Lottery 合約狀態
合约擁有者: 0x02f396aD6Ba37602441bE148AC2876ff595E2aea
入場費: 0.01 ETH (10000000000000000 wei)
參與者數量: 3
抽獎狀態: ✅ 進行中
獎金池: 0.03 ETH
合約餘額: 0.03 ETH

上一輪獲勝者: 0x46f03e16aEd64af7CC8fcFa221221E702a85bF1a
上一輪獎金: 0.03 ETH

查看參與者列表
參與者列表 (3 人):
  1. 0xF4A24c01ED89684A087254E88D97A0eaF99866E7
  2. 0x6F3019f6B620E03fDDB14bb6fFbB08a05263E8dd
  3. 0x46f03e16aEd64af7CC8fcFa221221E702a85bF1a


In [63]:
draw_lottery(official_account)


執行抽獎（僅合約擁有者）
準備執行抽獎，目前有 3 位參與者...
交易哈希: d03cd28437bf58b15f5f197c3e7794bada97aae3c662a7442248fb381a746b66
✅ 抽獎完成！交易已確認，區塊: 24880

🎉 獲勝者: 0x6F3019f6B620E03fDDB14bb6fFbB08a05263E8dd
💰 獎金: 0.03 ETH

新一輪狀態:
  參與者數量: 0
  獎金池: 0 ETH


In [64]:
for join_account in join_accounts:
    check_account_balance(join_account)

address:0xF4A24c01ED89684A087254E88D97A0eaF99866E7 balance: 0.999717996998025979 ETH
address:0x6F3019f6B620E03fDDB14bb6fFbB08a05263E8dd balance: 0.999813051998691364 ETH
address:0x46f03e16aEd64af7CC8fcFa221221E702a85bF1a balance: 0.999805506998638549 ETH
